In [3]:
#categorize all pixels in the bird's view into left group and right group
#fit left and right quadratic curve lines using detected pixels in left group and right group respectively

import cv2  
import numpy as np
import math


def find_lane(frame):
    y,x=frame.shape
    #print(frame.shape)
    nonzero=np.transpose(np.nonzero(frame))
    L=[m for m in nonzero if m[1]<0.5*x]
    R=[n for n in nonzero if n[1]>0.5*x]    
    #print('L',L)
    #print('R',R)
    return L,R

    
     

def fit_lanes(points_l, points_r, fit_globally=True):
    yl,xl=np.transpose(points_l)
    yr,xr=np.transpose(points_r)
    fit_ret={}
    if fit_globally:
        parameters = np.polyfit(yl, xl, 2)
        fit_ret.update({'al':parameters[0], 'bl':parameters[1],'cl':parameters[2],
                       'ar':parameters[0], 'br':parameters[1],'cr':parameters[2]})
    else:
        parameters_l= np.polyfit(yl, xl, 2)
        parameters_r= np.polyfit(yr, xr, 2)
        fit_ret.update({'al':parameters_l[0], 'bl':parameters_l[1],'cl':parameters_l[2],
                       'ar':parameters_r[0], 'br':parameters_r[1],'cr':parameters_r[2]})

    return fit_ret


def line_fit(binary_warped,fit_globally=True, quadratic=True):
    """
    Find and fit lane lines
    """
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = (np.dstack((binary_warped, binary_warped, binary_warped))*255).astype('uint8')
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[100:midpoint]) + 100
    rightx_base = np.argmax(histogram[midpoint:-100]) + midpoint
    #leftx_base = np.argmax(histogram[437-5:437+5]) + 437-5
    #rightx_base = np.argmax(histogram[800-5: 800+5]) + 800-5
    #leftx_base = 437
    #rightx_base = 800

    # Choose the number of sliding windows
    nwindows = 10
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix_l = 10
    minpix_r = 10
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - (window)*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,0,255), 3)
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,0,255), 3)
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix_l:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix_r:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    fit_ret={}
    if fit_globally:
        if quadratic:
            parameters_l = np.polyfit(lefty, leftx, 2)
            parameters_r= np.polyfit(righty, rightx, 2)
            fit_ret.update({'al':parameters_l[0], 'bl':parameters_l[1],'cl':parameters_l[2],
                            'ar':parameters_l[0], 'br':parameters_l[1],'cr':parameters_r[2]})
        else:
            parameters_l = np.polyfit(lefty, leftx, 3)
            parameters_r= np.polyfit(righty, rightx, 3)
            fit_ret.update({'al':parameters_l[0], 'bl':parameters_l[1],'cl':parameters_l[2],'dl':parameters_l[3],
                            'ar':parameters_l[0], 'br':parameters_l[1],'cr':parameters_l[2],'dr':parameters_r[3]})
    else:
        if quadratic:
            parameters_l = np.polyfit(lefty, leftx, 2)
            parameters_r= np.polyfit(righty, rightx, 2)
            fit_ret.update({'al':parameters_l[0], 'bl':parameters_l[1],'cl':parameters_l[2],
                            'ar':parameters_r[0], 'br':parameters_r[1],'cr':parameters_r[2]})
        else:
            parameters_l = np.polyfit(lefty, leftx, 3)
            parameters_r= np.polyfit(righty, rightx, 3)
            fit_ret.update({'al':parameters_l[0], 'bl':parameters_l[1],'cl':parameters_l[2],'dl':parameters_l[3],
                            'ar':parameters_r[0], 'br':parameters_r[1],'cr':parameters_r[2],'dr':parameters_r[3]})

    return fit_ret, out_img





def draw_lanes(frame,parameters,quadratic=True):
    pts_l=[]
    pts_r=[]
    y=np.linspace(0,frame.shape[0]-1,num=frame.shape[0])
    if quadratic:
        xl=parameters['al']*y**2+parameters['bl']*y+parameters['cl']
        xr=parameters['ar']*y**2+parameters['br']*y+parameters['cr']
    else:
        xl=parameters['al']*y**3+parameters['bl']*y**2+parameters['cl']*y+parameters['dl']
        xr=parameters['ar']*y**3+parameters['br']*y**2+parameters['cr']*y+parameters['dr']
    pts_l=np.array([np.transpose(np.vstack([xl,y]))])
    pts_r=np.array([np.flipud(np.transpose(np.vstack([xr,y])))])
    polygon = np.zeros_like(frame)
    pts = np.hstack((pts_l, pts_r))
    cv2.fillPoly(polygon, np.int_([pts]), (255,0,0))
    return polygon


#run the test video
cap = cv2.VideoCapture('project_video.mp4',0)
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('outcome_harder.avi',fourcc, 20.0, (1280,720))
frame_count = 1

while(cap.isOpened()):
#while True:  
    _,frame = cap.read()  
    #print('frame',frame.shape)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(3,3),0)
    edges_white=cv2.Canny(gray,200,250)
    
    HSL = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
    lower = np.uint8([ 10,   0,  100])
    upper = np.uint8([ 40, 255, 255])
    yellow_mask = cv2.inRange(HSL, lower, upper)
    YellowImg = cv2.bitwise_and(frame, frame, mask=yellow_mask)
    GrayImg = cv2.cvtColor(YellowImg, cv2.COLOR_BGR2GRAY)
    blurredImg = cv2.GaussianBlur(GrayImg, (3, 3), 0)
    edges_yellow = cv2.Canny(blurredImg, threshold1=100, threshold2=210)
    
    edges=cv2.addWeighted(edges_white, 1.0, edges_yellow, 1.0,0)
    
    
    original=frame.copy()
    n1,n2,n3=original.shape
    
    # implement homography transformation
    #src = np.float32([[237, 720],[1100, 720],[440, 450],[840, 450]])
    #dst = np.float32([[437, 720],[800, 720],[300, 0],[980, 0]])
    src = np.float32([[200, 720],[1100, 720],[590, 450],[685, 450]])
    dst = np.float32([[150, 720],[1020, 720],[300, 0],[980, 0]])
    world=np.array(src)
    ground=np.array(dst)
    h,status=cv2.findHomography(world,ground,cv2.RANSAC,5,maxIters=100, confidence=0.99)
    h_inv,status=cv2.findHomography(world,ground,cv2.RANSAC,5,maxIters=100, confidence=0.99)#h_inv for unwarpping the bird's view 
    projected_transformation=cv2.warpPerspective(original,h,(n2,n1),flags=cv2.INTER_LINEAR)
    projected_edges=cv2.warpPerspective(edges,h,(n2,n1),flags=cv2.INTER_LINEAR)
    

    
   
    parameters=line_fit(projected_edges,False,True)[0]
    algorithm=line_fit(projected_edges,False,True)[1]
    #points=find_lane(projected_edges)                                #find lanes in bird's view
    #parameters=fit_lanes(points[0], points[1], fit_globally=False)   #fit the quadratic curve model and get curve parameters
    #print('parameters=',parameters)
    polygon=draw_lanes(original,parameters,True)                          #draw curve lines using the calculated parameters
    
    
    
    #projected back to the world view
    h,status=cv2.findHomography(ground,world,cv2.RANSAC,5,maxIters=100, confidence=0.99)
    h_inv,status=cv2.findHomography(ground,world,cv2.RANSAC,5,maxIters=100, confidence=0.99)#h_inv for unwarpping the bird's view 
    bird_view=cv2.addWeighted(projected_transformation, 1.0, polygon, 0.5,0)
    polygon_back=cv2.warpPerspective(polygon,h,(n2,n1),flags=cv2.INTER_LINEAR)
    outcome=cv2.addWeighted(original, 1.0, polygon_back, 0.5,0)# add the polygon layer to the oringinal frame
    
    
    #cv2.imshow("canny",edges)
    cv2.imshow('projected edge', projected_edges)
    #cv2.imshow('projected transformation',projected_transformation)
    #cv2.imshow('polygon',polygon)
    cv2.imshow('bird view',bird_view)
    cv2.imshow('outcome',outcome)
    cv2.imshow('algorithm',algorithm)

    
    #out.write(outcome)
    
    params = []  
    #params.append(cv.CV_IMWRITE_PXM_BINARY)  
    params.append(1)  
    #cv2.imwrite("world" + "_%d.jpg" % frame_count, out, params)
    cv2.imwrite('algorithm' + "_%d.jpg" % frame_count, algorithm, params)
    #cv2.imwrite('bird view' + "_%d.jpg" % frame_count, bird_view, params)
    cv2.imwrite('outcome' + "_%d.jpg" % frame_count, outcome, params)
    frame_count = frame_count + 1  
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
cap.release()
out.release()
cv2.destroyAllWindows()